# 모델 매개변수 최적화하기
이제 모델과 데이터가 준비되었으니, 데이터에 매개변수를 최적화하여 모델을 학습하고 검증하고, 테스트할 차례이다.
모델을 학습하는 과정은 반복적인 과정을 거친다; 각 반복 단계에서 모델은 출력을 추측하고, 추측과 정답 사이의 오류를 계산하고, 매개변수에 대한 오류의 도함수를 수집한 뒤, 경사하강법을 사용하여 이 파라미터들을 최적화합니다.


## 기본(Pre-requisite) 코드
이전 장인 Dataset과 DataLoader와 신경말 모델 구성하기에서의 코드를 가져왔다.

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## 하이퍼파라미터(Hyperparameter)
하이퍼파라미터는 모델 최적하 과정에 제어할 수 있는 조절 가능한 매개변수이다. 
서로 다른 하이퍼파라미터 값은 모델 학습과 수렴율에 영향을 미칠 수 있다.   
학습 시에는 다음과 같은 하이퍼파라미터를 정의:
- 에폭(epoch) 수 : 데이터셋을 반복하는 횟수
- 배치크기(batch size) : 매개변수가 경신되기 전 신경망을 통해 전파된 데이터새믚ㄹ링 수
- 학습률(learning rate) : 각 배치.에폭에서 모델의 매개변수를 조절하는 비율. 값이 작을수록 학습속도가 느려지고, 값이 크며 학습 중 예측할 수 없는 동작이 발생할 수 있다.

In [3]:
learning_rate = 1e-3
batch_size  = 64
epochs = 5

## 최적화 단계(Optimization Loop)
하이퍼파라미터를 설정한 뒤에는 최적화 단계를 통해 모델을 학습하고 최적화 할 수 있다. 최적화 단계의 간 반복을 에폭이라고 부른다.  
하나의 에폭은 다음 두 부분으로 구성  
- 학습 단계 : 학습용 데이터셋을 반복하고 최적의 매개변수로 수렴
- 검증/테스트 단계 : 모델 성능이 개선되고 있는지를 확인하기 위해 테스트 데이터셋을 반복한다.  

학습 단계에서 일어나는 몇 가지 개념들을 간락히 살펴보자. 최적화 단계를 보려면 전체 구현 부분으로 건너뛰면 된다.

## 손실 함수(loss function)
학습용 데이터를 제공하면, 학습되지 않은 신경망은 정답을 제공하지 않을 확률이 높다. 손실함수는 획득한 결과에 실제 값 사이의 틀린 정도를 측정하며, 학습 중에 이값을 최소화하려고 한다.  
주어진 데이터 샘플을 이렵으로 계산한 예측과 정답을 비교하여 손실을 계산한다.  

일반적으로 손실함수에는 회귀문제에 사용되는 nn.MSELoss(평균제곱오차)나 분류에 사용되는 nn.NLLLoss(음의 로그 우도),그리고 nn.LogSoftmax와 nn.NLLLoss를 합친 nn.CrossEntropyLoss 등이 있다. 

모델의 출력 로짓을 nn.CrossEntropyLoss에 전달하여 로짓을 정규화하고 예측 오류를 계산 한다. 

In [5]:
# 손실 함수를 초기화한다.
loss_fn = nn.CrossEntropyLoss()

## 옵티마이저(Optimizer)
최적화는 각 학습 단계에서 모델의 오류를 줄이기 위해 모델 매개변수를 조정하는 과정이다. 최적화 알고리즘은 이 과정이 수행되는 방식(여기에서는 확률적 경사하강법(SGD))을 정의한다. 모든 최적화 절차는 optimizer 객체에 캡술화된다. 여기서는 SGD 옵티마이저를 사용하고 있으며, Pytorch에는 ADAM이나 RMSProp과 같은 다른 종류의 모델과 데이터에서 더 잘 동작하는 다양한 옵티마이저가 있다.  

학습하려는 모델의 매개변수와 학습률 하이퍼파라미터를 등록하여 옵티마이저를 초기화합니다.

In [6]:
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

학습 단계에서 최적화는 세단계로 이뤄진다:  
- optimizer.zero_grad()를 호출하여 모델 매개변수의 변화도를 재설정한다. 기본적으로 변화도는 더해지기(add up) 때문에 중복 계산을 막기 위해 반복할 때마다 명시적으로 0으로 설정한다.
- loss.backward()를 호출하여 예측손실을 역전파한다. PyTorch는 각 매개변수에 대한 손실의 변화도를 저장한다. 
- 변화도를 계산한 뒤에는 optimizer.step()을 호출하여 역전파 단계에서 수집된 변화도로 매개변수를 조정한다.    

# 전체 구현
최적화 코드를 반복하여 수행하는 train_loop와 데스트 데이터로 모델의 성능을 측정하는 test_loop를 정의한다.

In [17]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 예측(prediction)과 손실(loss) 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

손실함수와 옵티마이저를 초기화하고 train_loop와 test_loop에 전달한다. 모델의 성능 향성을 알아보기 위해 자유롭게 에폭 수를 증가시켜 볼 수 있다.

In [18]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.305048  [   64/60000]
loss: 2.294462  [ 6464/60000]
loss: 2.274365  [12864/60000]
loss: 2.267047  [19264/60000]
loss: 2.244561  [25664/60000]
loss: 2.221665  [32064/60000]
loss: 2.221543  [38464/60000]
loss: 2.183487  [44864/60000]
loss: 2.177596  [51264/60000]
loss: 2.156495  [57664/60000]
Test Error: 
 Accuracy: 50.1%, Avg loss: 2.145827 

Epoch 2
-------------------------------
loss: 2.153982  [   64/60000]
loss: 2.148807  [ 6464/60000]
loss: 2.090475  [12864/60000]
loss: 2.104820  [19264/60000]
loss: 2.047405  [25664/60000]
loss: 1.998719  [32064/60000]
loss: 2.011479  [38464/60000]
loss: 1.930349  [44864/60000]
loss: 1.924637  [51264/60000]
loss: 1.868450  [57664/60000]
Test Error: 
 Accuracy: 60.2%, Avg loss: 1.863125 

Epoch 3
-------------------------------
loss: 1.889844  [   64/60000]
loss: 1.868286  [ 6464/60000]
loss: 1.748645  [12864/60000]
loss: 1.789191  [19264/60000]
loss: 1.673789  [25664/60000]
loss: 1.640881  [32064/600